# Preprocesamiento de Datos #
La calidad de los datos y la cantidad de información útil que contiene son factores clave que determinan lo bien que puede aprender un algoritmo de aprendizaje automático. Por lo tanto, es absolutamente crítico asegurarse de que se examina y se preprocesa el conjunto de datos antes de alimentarlo a un algoritmo de aprendizaje.

Los temas que se van a ver son los siguientes:
- Eliminar e introducir valores perdidos del conjunto de datos
- Dar forma a datos categóricos poder utilizarlos en algoritmos de aprendizaje automático
- Seleccionar características relevantes para la construcción del modelo

## Gestión de datos faltantes ##
No es raro en las aplicaciones del mundo real que a los ejemplos de entrenamiento les falten uno o más valores por diversas razones. Desafortunadamente, la mayoría de las herramientas computacionales no pueden manejar esos valores perdidos o producirán resultados impredecibles si simplemente se ignoran. Por ello, es crucial que se traten esos valores faltantes antes de proceder con análisis posteriores.

### Identificación de valores faltantes en datos tabulares ###
Primero se crean datos de ejemplo con valores faltantes

In [1]:
import pandas as pd
from io import StringIO

In [3]:
csv_data = '''A,B,C,D
         1.0,2.0,3.0,4.0
         5.0,6.0,,8.0
         10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


Para DataFrame grande, puede ser tedioso buscar valores faltantes manualmente; en estos casos, se puede utilizar el método `isnull` para devolver un `DataFrame` con valores booleanos que indican si una celda contiene un valor numérico (`False`) o si faltan datos (`True`). Usando el método `sum`, se devuelve el número de valores faltantes por columna de la siguiente manera:

In [4]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

> ##### Manejo conveniente de datos con DataFrames de pandas #####
> Aunque scikit-learn se desarrolló originalmente para trabajar solo con matrices NumPy, a veces puede ser más conveniente preprocesar datos utilizando el `DataFrame` de pandas. Hoy en día, la mayoría de las funciones de scikit-learn admiten objetos `DataFrame` como parámetro, pero dado que el manejo de la matriz NumPy es más maduro en la API scikit-learn, se recomienda usar matrices NumPy cuando sea posible. Hay que tener en cuenta que siempre se puede acceder a la matriz subyacente NumPy de un `DataFrame` a través del atributo `values` antes de pasarlo a un estimador scikit-learn:

In [5]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

### Eliminar ejemplos o características de entrenamiento con valores faltantes ###
Una de las formas más fáciles de lidiar con los datos faltantes es simplemente eliminar por completo las características (columnas) o ejemplos de entrenamiento (filas) correspondientes del conjunto de datos; Las filas con valores perdidos pueden eliminarse fácilmente mediante el método `dropna`, pasandolo el valor $0$ (el indice de la fila) al parámetro `axis`:

In [6]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


Del mismo modo, se puede eliminar columnas que tengan al menos un NaN en cualquier fila estableciendo el argumento de `axis` a $1$:

In [7]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


El método `dropna` admite varios parámetros adicionales que pueden ser útiles:
- Solo elimina las filas donde todas las columnas son NaN

In [8]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


- Elimina las filas que tienen menos de 4 valores reales

In [9]:
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


- Solo se eliminan las filas donde NaN aparezca en columnas específicas (aquí: 'C')

In [10]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


Aunque la eliminación de datos faltantes parece ser un enfoque conveniente, también conlleva ciertas desventajas; por ejemplo, se puede terminar eliminando demasiadas muestras, lo que hará imposible un análisis confiable. O, si se eliminan demasiadas columnas de características, se corre el riesgo de perder información valiosa que el clasificador necesita para discriminar entre clases.

### Introducir valores faltantes ###
Cuando no es posible eliminar columnas de características porque se perderión datos valiosos, se puede utilizar diferentes técnicas de interpolación para estimar los valores faltantes a partir de otros ejemplos de entrenamiento del conjunto de datos. 

Una de las técnicas de interpolación más comunes es la **introducción de la media**, donde simplemente reemplazamos el valor faltante con el valor medio de toda la columna de características. Una forma conveniente de lograr esto es mediante el uso de la clase `SimpleImputer` de scikit-learn:

In [11]:
from sklearn.impute import SimpleImputer
import numpy as np

In [12]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Otras opciones para el parámetro de estrategia son `median` o `most_frequent`, que reemplaza los valores faltantes con los valores más frecuentes. Esto es útil para introducir valores categóricos de características, por ejemplo, una columna de características que almacena una codificación de nombres de colores, como rojo, verde y azul.

Alternativamente, una forma aún más conveniente de introducir valores perdidos es utilizando el método `fillna` de pandas y proporcionando un método de entrada como argumento. Por ejemplo, usando pandas, se lograría la misma entrada de la media directamente en el objeto `DataFrame`

In [13]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### Comprender el API estimador de scikit-learn ###
La clase `SimpleImputer` pertenece a las llamadas clases **transformador** en scikit-learn, que se utilizan para la transformación de datos. Los dos métodos esenciales de esos estimadores son `fit` y `transform`. El método de `fit` se usa para aprender los parámetros de transformación a partir de los datos de entrenamiento, y el método de `transform` usa esos parámetros para transformar los datos. Cualquier matriz de datos que se vaya a transformar debe tener el mismo número de características que la matriz de datos que se utilizó para adaptarse al modelo.

Es transformador se tiene que utilizar para transformar tanto el conjunto de datos de entrenamiento como el conjunto de datos de test.

![flujo transformador](imgs/flujo_transform.png)

Los clasificadores que se han utilizado hasta ahora pertenecen a los llamados **estimadores** en scikit-learn, con una API que es conceptualmente muy similar a los de las clases transformador. Los estimadores tienen un método `prediction` pero también pueden tener un método `transform`, que se verá más adelante. Como se recordará, también se utiliza el método `fit` para aprender los parámetros de un modelo cuando se entrena a estos estimadores para la clasificación. Sin embargo, en las tareas de aprendizaje supervisado, también proporcionamos las etiquetas de clase para ajustar el modelo, que luego se pueden utilizar para hacer predicciones sobre nuevos ejemplos de datos no etiquetados a través del método `predict`, como se ilustra en la siguiente figura:

![flujo estimador](imgs/flujo_estimador.png)